In [13]:
import requests
import pandas as pd
import json


# Parameters for API call
offset = 0
length = 100

url = f"https://datasets-server.huggingface.co/rows?dataset=KodCode%2FKodCode-V1-SFT-4o&config=default&split=train&offset={offset}&length={length}"
response = requests.get(url)

# Convert to DataFrame
rows = response.json()['rows']
data = pd.DataFrame([row['row'] for row in rows])
data

,style,subset,question_id,question,solution,test_code,test_info,gpt_pass_sequence,gpt_pass_trial_num,gpt_difficulty,gpt_pass_percentage,4o_pass_sequence,4o_pass_trial_num,4o_correctness,4o_solution,metadata,conversations
0,instruct,Leetcode,Leetcode_3_I,"Given a list of integers `nums`, find the maxi...","def max_product(nums):\n """"""\n Returns t...",from solution import max_product\n\ndef test_m...,[{'docstring': 'Returns the maximum product of...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",1,hard,0.1,"[1, 0, 1]",2,True,def max_product(nums):\n # Sort the list to...,"{'original_instruction': '', 'prompt_id': '000...","[{'from': 'human', 'value': 'Given a list of i..."
1,instruct,Leetcode,Leetcode_8_I,"Given a list of integers `nums`, consider a li...","def min_subarray_len(target, nums):\n """"""\n...",from solution import min_subarray_len\n\ndef t...,[{'docstring': 'Returns the length of the shor...,"[0, 1, 0, 1, 0, 0, 0, 0, 0, 0]",2,hard,0.2,"[1, 1, 1]",3,True,"def min_subarray_len(target, nums):\n n = l...","{'original_instruction': '', 'prompt_id': '000...","[{'from': 'human', 'value': 'Given a list of i..."
2,instruct,Leetcode,Leetcode_11_I,"Given an array of integers `arr`, you are to f...","def longest_distinct_subsequence(arr):\n """"...",from solution import longest_distinct_subseque...,[{'docstring': 'Returns the length of the long...,"[1, 1, 1, 0, 1, 1, 1, 1, 0, 1]",8,easy,0.8,"[1, 0, 1]",2,True,def longest_distinct_subsequence(arr):\n se...,"{'original_instruction': '', 'prompt_id': '000...","[{'from': 'human', 'value': 'Given an array of..."
3,instruct,Leetcode,Leetcode_12_I,A tropical island is represented by a grid of ...,"def max_coconuts(coconuts):\n """"""\n Find...",from solution import max_coconuts\n\ndef test_...,[{'docstring': 'Find the maximum number of coc...,"[0, 0, 0, 1, 0, 0, 0, 1, 0, 0]",2,hard,0.2,"[1, 0, 0]",1,True,def max_coconuts(coconuts):\n if not coconu...,"{'original_instruction': '', 'prompt_id': '000...","[{'from': 'human', 'value': 'A tropical island..."
4,instruct,Leetcode,Leetcode_24_I,"Given an integer `n`, return the list of prime...","def sieve_of_eratosthenes(n):\n """"""\n Re...",from solution import sieve_of_eratosthenes\n\n...,[{'docstring': 'Returns a list of all prime nu...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",10,easy,1.0,"[1, 1, 1]",3,True,def sieve_of_eratosthenes(n):\n if n <= 2:\...,"{'original_instruction': '', 'prompt_id': '000...","[{'from': 'human', 'value': 'Given an integer ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,instruct,Leetcode,Leetcode_465_I,Given a string `s` which consists of lowercase...,"def reverse_only_letters(s):\n """"""\n Rev...",from solution import reverse_only_letters\n\nd...,[{'docstring': 'Reverses only the letters in t...,"[1, 1, 0, 1, 1, 1, 1, 1, 1, 1]",9,easy,0.9,"[1, 1, 1]",3,True,def reverse_only_letters(s: str) -> str:\n ...,"{'original_instruction': '', 'prompt_id': '000...","[{'from': 'human', 'value': 'Given a string `s..."
96,instruct,Leetcode,Leetcode_466_I,Write a function `longestConsecutiveSubsequenc...,def longestConsecutiveSubsequence(nums):\n ...,from solution import longestConsecutiveSubsequ...,[{'docstring': 'Returns the length of the long...,"[0, 1, 1, 0, 0, 0, 1, 1, 1, 1]",6,medium,0.6,"[1, 1, 1]",3,True,def longestConsecutiveSubsequence(nums):\n ...,"{'original_instruction': '', 'prompt_id': '000...","[{'from': 'human', 'value': 'Write a function ..."
97,instruct,Leetcode,Leetcode_467_I,A company has a list of `n` projects labeled f...,"from collections import defaultdict, deque\n\n...",from solution import find_order\n\ndef test_fi...,[{'docstring': 'Determine the order in which p...,"[1, 1, 1, 0, 1, 0, 0, 0, 1, 1]",6,medium,0.6,"[0, 0, 1]",1,True,"from collections import deque, defaultdict\n\n...","{'original_instruction': '', 'prompt_id': '000...","[{'from': 'human', 'value': 'A company has a l..."
98,instruct,Leetcode,Leetcode_471_I,"You are given a list of `tasks`, where each ta...","from collect

## **Column Name	Description**

solution	--> The function implementation

test_code	--> Unit tests for that solution

question	--> Natural-language task

4o_solution	--> GPT-4o generated solution

gpt_pass_percentage	--> How often GPT passed it (difficulty proxy)

4o_correctness	--> Whether GPT-4o’s solution is correct

style --> instruct& complete

subset --> 	Metadata on task origin (e.g. Leetcode)

now TRY JSON

TEST:


In [15]:
import requests
import random
import re
from sklearn.model_selection import train_test_split
import os
from openai import OpenAI
import getpass
import json
from dotenv import load_dotenv
import os
import openai

load_dotenv()
openai.api_key = os.getenv("FIREWORKS_API_KEY")
openai.api_base = "https://api.fireworks.ai/inference/v1"

model_id = "accounts/fireworks/models/deepseek-v3"

# 1. Download KodCode train split rows (first 100 for demo)
def download_kodcode_train_rows(offset=0, length=100):
    """Download a subset of KodCode dataset from Hugging Face."""
    url = (
        f"https://datasets-server.huggingface.co/rows?"
        f"dataset=KodCode%2FKodCode-V1-SFT-4o&config=default&split=train&offset={offset}&length={length}"
    )
    resp = requests.get(url)
    resp.raise_for_status()
    data = resp.json()
    return data["rows"]

# 2. Extract style features from solution code
def extract_advanced_style_features(solution_code):
    lines = solution_code.strip().split("\n")
    func_len = len(lines)

    code_no_comments = "\n".join([line for line in lines if not line.strip().startswith("#")])
    code_no_strings = re.sub(r'(\".*?\"|\'.*?\')', '', code_no_comments)

    var_names = re.findall(r"\b([a-zA-Z_][a-zA-Z0-9_]*)\b", code_no_strings)
    keywords = {
        'def','return','if','else','for','while','in','and','or','not','True','False','None',
        'import','from','as','class','with','try','except','finally','break','continue','pass',
        'lambda','print','yield','assert','async','await','global','nonlocal','raise'
    }
    var_names = [v for v in var_names if v not in keywords]

    snake_case = sum(1 for v in var_names if "_" in v)
    camel_case = sum(1 for v in var_names if re.search(r"[a-z]+[A-Z]", v))

    indent_levels = []
    for line in lines:
        stripped = line.lstrip('\t ')
        indent = len(line) - len(stripped)
        indent_levels.append(indent)
    max_nesting = max(indent_levels) if indent_levels else 0

    comment_lines = sum(1 for line in lines if "#" in line.strip())
    comment_density = comment_lines / func_len if func_len else 0

    branches = ['if', 'elif', 'else', 'for', 'while', 'and', 'or', 'except', 'case']
    complexity_count = sum(len(re.findall(rf"\\b{kw}\\b", line)) for line in lines for kw in branches)


    return {
        "function_length": func_len,
        "var_count": len(set(var_names)),
        "snake_case_count": snake_case,
        "camel_case_count": camel_case,
        "max_nesting_depth": max_nesting,
        "comment_density": comment_density,
        "cyclomatic_complexity": complexity_count,
    }

# Helper: parse bullet points from LLM output text
def parse_bullets(text):
    lines = text.strip().split("\n")
    bullets = []
    for line in lines:
        line = line.strip()
        if line.startswith("- ") or line.startswith("• ") or line.startswith("* "):
            bullets.append(line[2:].strip())
    # If no bullets found, just return the whole text as one string in list
    if not bullets:
        return [text.strip()]
    return bullets


# 3. Generate dynamic recommendations using OpenAI (new SDK)
def recommend_with_llm(solution_code, features):
    system_prompt = """You are a Python code style reviewer.
Given a function and extracted style metrics, make concrete suggestions for improving style, readability, and maintainability. Output 3–5 recommendations as bullet points.
If the code is excellent, return 'No improvements needed.'"""

    # Truncate code if too long
    if len(solution_code.split('\n')) > 50:
        solution_code = '\n'.join(solution_code.split('\n')[:50]) + "\n# ... truncated ..."

    user_prompt = f"""
Code:
{solution_code}

Extracted features:
- Function length: {features['function_length']}
- Variable count: {features['var_count']}
- Snake case variables: {features['snake_case_count']}
- Camel case variables: {features['camel_case_count']}
- Max nesting depth: {features['max_nesting_depth']}
- Comment density: {features['comment_density']:.2f}
- Cyclomatic complexity (proxy): {features['cyclomatic_complexity']}

What are the recommendations?
"""

    try:
        client = OpenAI(
            api_key=openai.api_key,
            base_url=openai.api_base
        )
        completion = client.chat.completions.create(
            model=model_id, # Use the qwen model
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.3,
        )
        raw_text = completion.choices[0].message.content.strip()
        print("🧠 Raw LLM output:\n", raw_text) # Log raw output
        bullets = parse_bullets(raw_text)

        # Return JSON/dict format for production use
        return {
            "recommendations": bullets,
            "raw_text": raw_text
        }
    except Exception as e:
        return {
            "error": f"LLM generation failed: {e}"
        }

def compare_with_ground_truth(row, llm_recommendations, features, sample_index):
    gt_quality = row["row"].get("gpt_pass_percentage", None)

    if "error" in llm_recommendations:
        print(f"[ERROR] {llm_recommendations['error']}")

    result = {
        "code_analysis": {
            "function_length": features['function_length'],
            "var_count": features['var_count'],
            "snake_case_count": features['snake_case_count'],
            "camel_case_count": features['camel_case_count'],
            "max_nesting_depth": features['max_nesting_depth'],
            "comment_density": round(features['comment_density'], 2),
            "cyclomatic_complexity": features['cyclomatic_complexity'],
        },
        "ground_truth_score": gt_quality,
        "recommendations": llm_recommendations.get("recommendations", []),
        "llm_raw_text": llm_recommendations.get("raw_text", ""),
        "error": llm_recommendations.get("error", None),
    }

    print(json.dumps(result, indent=2))

    with open(f"recommendation_sample_{sample_index}.json", "w") as f:
        json.dump(result, f, indent=2)



# Main driver function
def main():
    print("Downloading KodCode sample rows...")
    rows = download_kodcode_train_rows(length=20)

    train_rows, test_rows = train_test_split(rows, test_size=0.2, random_state=42)
    print(f"Train samples: {len(train_rows)}, Test samples: {len(test_rows)}")

    for i, row in enumerate(test_rows):
        print(f"\n=== Test sample {i+1} ===")
        solution_code = row["row"]["solution"]
        features = extract_advanced_style_features(solution_code)
        print("Extracted features:", features)
        llm_recommendations = recommend_with_llm(solution_code, features)
        compare_with_ground_truth(row, llm_recommendations, features, i + 1)

if __name__ == "__main__":
    main()

Train samples: 16, Test samples: 4

=== Test sample 1 ===
Extracted features: {'function_length': 9, 'var_count': 14, 'snake_case_count': 1, 'camel_case_count': 0, 'max_nesting_depth': 4, 'comment_density': 0.2222222222222222, 'cyclomatic_complexity': 0}
🧠 Raw LLM output:
 The code is already well-written and adheres to good style practices. However, here are a few minor suggestions for further improvement:

- **Add a brief example in the docstring**: Including an example in the docstring can make it clearer how the function should be used. For instance:  
  ```python
  """
  Returns the maximum product of any two distinct elements in the list.

  Example:
  >>> max_product([1, 3, -5, -2])
  15
  """
  ```

- **Consider adding type hints**: Adding type hints can improve readability and help with static type checking. For example:  
  ```python
  def max_product(nums: list[int]) -> int:
  ```

- **Clarify the comment about sorting**: The comment `# Sort the list` is straightforward, but